In [34]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math as mt
import os
import pickle
import mlflow
import mlflow.keras

In [3]:
# print("numpy=="+np.__version__)
# print("tensorflow=="+tf.__version__)
# print("pandas=="+pd.__version__)
# print("mlflow=="+mlflow.__version__)


AttributeError: module 'os' has no attribute '__version__'

In [35]:
bs=10
epochs=150

In [36]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

Physical devices cannot be modified after being initialized


In [37]:



mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("graph-neural-network-with-3-channel")

2022/06/27 13:54:44 INFO mlflow.tracking.fluent: Experiment with name 'graph-neural-network-with-3-channel' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/6', experiment_id='6', lifecycle_stage='active', name='graph-neural-network-with-3-channel', tags={}>

In [38]:
mlflow.tensorflow.autolog()

In [ ]:
# def adjm(h,w,near):
#   am=[[0 for i in range(w*h)] for j in range(h*w)]
#   img=[[0 for i in range(w)] for j in range(h)]
  
#   dr=[[1,0],[0,1],[1,1],[-1,-1],[-1,0],[0,-1],[-1,1],[1,-1]]

#   for i,v in enumerate(img):
#     for j,v2 in enumerate(img[i]):
#       timg=[[0 for i in range(w)] for j in range(h)]
#       for k in range(0,near+1):
#         for d in dr:
#           posy=i+d[0]*k
#           posx=j+d[1]*k
#           if posy>=0 and posy<h and posx>=0 and posx<w:
#             timg[posy][posx]=1
#             am[i*w+j][posy*w+posx]=1
#         print(timg)  

#   return am

# val=adjm(4,5,1)

In [39]:
with open('data/train/x_train.pkl','rb') as f:
        x1=pickle.load(f)
with open('data/train/y_train.pkl','rb') as f:
        y1=pickle.load( f)
with open('data/valid/x_valid.pkl','rb') as f:
        x2=pickle.load(f)
with open('data/valid/y_valid.pkl','rb') as f:
        y2=pickle.load( f)
# with open('data/test/x_test.pkl','rb') as f:
#         x3=pickle.load(f)
# with open('data/test/y_test.pkl','rb') as f:
#         y3=pickle.load( f)

In [40]:
with open('data/test/x_test.pkl','rb') as f:
        x3=pickle.load(f)
with open('data/test/y_test.pkl','rb') as f:
        y3=pickle.load( f)

In [7]:
x1.shape

NameError: name 'x1' is not defined

In [9]:
print(max(y1))

6.0


In [41]:
class GCN(tf.keras.layers.Layer):
    def __init__(self,inp_shape=9*12,out_shape=9*12,h=9, w=12,near=3):
        super(GCN, self).__init__()
        self.am=[[0 for i in range(w*h)] for j in range(h*w)]
        self.img=[[0 for i in range(w)] for j in range(h)]
  
        self.dr=[[1,0],[0,1],[1,1],[-1,-1],[-1,0],[0,-1],[-1,1],[1,-1]]

        for i,v in enumerate(self.img):
           for j,v2 in enumerate(self.img[i]):
        # timg=[[0 for i in range(w)] for j in range(h)]
              for k in range(0,near+1):
                for d in self.dr:
                  posy=i+d[0]*k
                  posx=j+d[1]*k
                  if posy>=0 and posy<h and posx>=0 and posx<w:
            # timg[posy][posx]=1
                    self.am[i*w+j][posy*w+posx]=1
        # print(timg) 
        self.am=np.array(self.am) 
        self.D = np.diag(np.sum(self.am, axis=0)) 
        self.Dinv=np.linalg.inv(self.D)
        self.am=tf.Variable(self.am,trainable=True,dtype=tf.float32,name="ajm")
        self.Dinv=tf.Variable(self.Dinv,trainable=True,dtype=tf.float32,name="dm")
        # self.norm=tf.matmul(self.Dinv*self.am)
        self.w = self.add_weight(
            shape=(inp_shape, out_shape), initializer="random_normal", trainable=True,dtype=tf.float32,name='adjw'
        )
        # self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)
    
    def call(self, inputs):
        inputs=tf.cast(inputs,dtype=tf.float32)
        # inputs=tf.transpose()
        # print(inputs.shape)
        # print("inpshape",tf.transpose(inputs,perm=[1,0]).shape)
        # print(self.Dinv.shape)
        # print(self.am.shape)
        # print(self.w.shape)
        first_half=tf.matmul(inputs,tf.matmul(self.Dinv,self.am))
    
        return tf.matmul(first_half, self.w) 

In [46]:
def build_model(near=3,lay=2,bs=bs,n_out=7):
    

    inputs = tf.keras.Input( shape=(9,12,3))
    input=tf.transpose(inputs,perm=[3,0,1,2])
    mlflow.log_param("layers", lay)
    mlflow.log_param("nearest", near)
    mlflow.set_tag("layer", "changed")
    r=input[0][:][:][:]
    g=input[1][:][:][:]
    b=input[2][:][:][:]
    # print("red",r.shape)
    r=tf.keras.layers.Flatten()(r)
    g=tf.keras.layers.Flatten()(g)
    b=tf.keras.layers.Flatten()(b)
    # print(inputs.shape)
   
    # x=tf.reshape(inputs,[inputs.shape[1]*inputs.shape[2]],name="initial reshape")
    # x=tf.keras.layers.Flatten()(inputs)
    for i in range(lay):
        r=GCN(near=near)(r)
        r=tf.keras.layers.Activation('relu',name="graph_layerr"+str(i+1))(r)
    r=tf.keras.layers.BatchNormalization()(r)
    for i in range(lay):
        g=GCN(near=near)(g)
        g=tf.keras.layers.Activation('relu',name="graph_layerg"+str(i+1))(g)
    g=tf.keras.layers.BatchNormalization()(g)
    
    for i in range(lay):
        b=GCN(near=near)(b)
        b=tf.keras.layers.Activation('relu',name="graph_layerb"+str(i+1))(b)
    b=tf.keras.layers.BatchNormalization()(b)
    x=tf.stack([r,g,b])
    x=tf.transpose(x,perm=[1,2,0])
    # print("x_shape_b",x.shape)
    x=tf.keras.layers.Dense(units=1,activation="relu",name="denseshrink")(x)
    # print("x_shape_a",x.shape)
    x=tf.squeeze(x,axis=-1)
    # print("x_shape",x.shape)
    x=tf.keras.layers.Dense(units=32,activation="relu",name="dense1")(x)
    output=tf.keras.layers.Dense(n_out,activation="softmax",name="out_layer")(x)
    model = tf.keras.Model(inputs=inputs, outputs=output, name="graph_neuralnetwork")

    model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=["accuracy"],run_eagerly=True)
    return model

In [47]:
model=build_model()

(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)


In [48]:
model.summary()

Model: "graph_neuralnetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 9, 12, 3)]   0           []                               
                                                                                                  
 tf.compat.v1.transpose_16 (TFO  (3, None, 9, 12)    0           ['input_18[0][0]']               
 pLambda)                                                                                         
                                                                                                  
 tf.__operators__.getitem_156 (  (None, 9, 12)       0           ['tf.compat.v1.transpose_16[0][0]
 SlicingOpLambda)                                                ']                               
                                                                                

In [49]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [24]:
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="mode/base_model",
#                                                  save_weights_only=True,
#                                                  verbose=1)

In [50]:
# with mlflow.start_run(nested=true):
history = model.fit(x1,y1, batch_size=bs, epochs=15, callbacks=[callback],validation_data=(x2,y2),verbose=2)


Layer GCN has arguments ['self', 'inp_shape', 'out_shape', 'h', 'w', 'near']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config
Epoch 1/15
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)


2022-06-27 13:59:40.323633: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)
(10, 108)


2022-06-27 14:00:09.473124: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
INFO:tensorflow:Assets written to: /var/folders/v9/13sg2_p569zdm3chj4w3dmr00000gn/T/tmpsub_i319/model/data/model/assets


In [51]:
model.save('/Users/ariqrahman/Desktop/Mac-Neural-Network-Project/model/base_model/gcn1')

(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
(None, 108)
INFO:tensorflow:Assets written to: /Users/ariqrahman/Desktop/Mac-Neural-Network-Project/model/base_model/gcn1/assets


In [52]:
model1=tf.keras.models.load_model("model/base_model/gcn1")

In [53]:
model1.summary()

Model: "graph_neuralnetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 9, 12, 3)]   0           []                               
                                                                                                  
 tf.compat.v1.transpose_16 (TFO  (3, None, 9, 12)    0           ['input_18[0][0]']               
 pLambda)                                                                                         
                                                                                                  
 tf.__operators__.getitem_156 (  (None, 9, 12)       0           ['tf.compat.v1.transpose_16[0][0]
 SlicingOpLambda)                                                ']                               
                                                                                